In [1]:
from metric_learn import Covariance
from visiprog.data import read_VSP_label, read_material_label, get_raw_feature
from visiprog.metric_learn import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG

from IPython.display import Image, display

np.set_printoptions(precision=2)

In [11]:
raw_feature = get_raw_feature()
print(raw_feature.shape)
material_label = read_material_label()
print(label)

(5245, 82)
1.0


# Baseline approach
STSIM-M approach, no training

In [12]:
cov = Covariance()
cov.fit(raw_feature)
X_STSIMM = cov.transform(raw_feature)

LinAlgError: Matrix is not positive definite

In [9]:
# np.savetxt('STSIM_M.csv', M, fmt='%.2f', delimiter=',')
# res = leave_one_sample_out(X_STSIMM)
# print(res['accuracy'])

# Fisher linear discriminant approach

Exact solution, however trained with semantic labels ...

In [13]:
model = LFDA()
model.fit(raw_feature, material_label)
X_fisher = model.transform(raw_feature)

In [14]:
# np.savetxt('FDA_semantic.csv', M, fmt='%.2f', delimiter=',')
res_fda = leave_one_sample_out(X_fisher, material_label)
print(res_fda['accuracy'])

0.989513822688


# Train using ViSiProg label

In [15]:
groups, N = read_VSP_label()

In [16]:
visiprog_label = np.zeros((N,))

count = 0
for g in groups:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if visiprog_label[i] != 0:
            label = visiprog_label[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        visiprog_label[i] = label


In [17]:
model = LFDA_VISIPROG()
model.fit(raw_feature, visiprog_label)
X_vsp = model.transform(raw_feature)

In [18]:
res_vsp = leave_one_sample_out(X_vsp, material_label)
print(res_vsp)

{'accuracy': 0.98970448045757864, 'prediction': array([21,  0,  0, ..., 60, 60, 60])}


In [19]:
L = model.transformer()
np.save("trained_L.npy",L)

In [ ]:
comparison = (res_vsp['prediction'] != res_fda['prediction'])
print(np.sum(comparison))

In [ ]:
mismatch = np.where(comparison == True)[0].astype(int)

In [ ]:
listImg = readImgList()


for i in mismatch:
    display(Image(filename = listImg[i]))
    gt = listImg[i].split("-")[1]
    print("Groundtruth {} prediction {}".format(listImg[i], int(res_vsp['prediction'][i])))
    

# Cross validation across materials